In [ ]:
import gensim

In [ ]:
import sklearn

In [ ]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import numpy
import utils

# random
#from random import shuffle
# classifier
#from sklearn.linear_model import LogisticRegression

In [ ]:
import numpy as np
en = np.loadtxt(open("/Users/hsajjad/Work/work/comparable-corpus/script/en-europarl.csv", "rb"), delimiter=",")
en=np.nan_to_num(en)

In [ ]:
print (en.shape)

In [ ]:
fr = np.loadtxt(open("/Users/hsajjad/Work/work/comparable-corpus/script/fr-europarl.csv", "rb"), delimiter=",")
fr=np.nan_to_num(fr)

In [ ]:
print (fr.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [ ]:
fr_chunk = fr[0:1]
en_chunk = en[0:21000]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(fr_chunk, en_chunk)
result.shape

In [ ]:
fr_chunk[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Change chunk_size to control resource consumption and speed
# Higher chunk_size means more memory/RAM needed but also faster 
chunk_size = 500 
matrix_len = en.shape[0] # Not sparse numpy.ndarray

def similarity_cosine_by_chunk(start, end):
    if end > matrix_len:
        end = matrix_len
    return cosine_similarity(X=en[start:end], Y=fr) # scikit-learn function

i =0

for chunk_start in range(0, matrix_len, chunk_size):
    f = "output" + "-" + str(i) + ".csv"
    
    cosine_similarity_chunk = similarity_cosine_by_chunk(chunk_start, chunk_start+chunk_size)
    print (cosine_similarity_chunk.shape)
    #np.savetxt(f, cosine_similarity_chunk, delimiter = ',')
    print (type(cosine_similarity_chunk))
    i +=1
    
    
    # Handle cosine_similarity_chunk  ( Write it to file_timestamp and close the file )
    # Do not open the same file again or you may end up with out of memory after few chunks

In [99]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from numpy import genfromtxt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import scipy.spatial as sp
import sys
from scipy import sparse


In [2]:
# load the file with cosine similarity scores
a=np.load('/Users/hsajjad/Work/work/comparable-corpus/output-0.csv.npy')

In [3]:
a.shape
print (np.argmax(a[0]))
a[2].max()

82788


0.66106670517620991

In [5]:
n=1
sorted_idx = np.argsort(-a) # get the sorted index ascending order based on values in the array
top_idx = sorted_idx[:,:n]

In [6]:
sorted_idx.shape

(500, 271874)

In [49]:
# this function load predicted pairs from the embedding matrix , one best pair
def pred_pairs(top_idx):
    pred_pair = {}
    for i in range(top_idx.shape[0]):
            src_sent = i + 1
            tgt_sent = top_idx[i,0] + 1
            pred_pair [str(src_sent) + "\t" + str(tgt_sent)] = 0
    return pred_pair

In [100]:
def load_prediction(fname):
    fin = open (fname, 'r', encoding='utf-8')
    pred = {}
    for line in fin:
        line = line.rstrip()
        pred[line] = 0 # loading french english pairs
    return (pred)

In [101]:
def load_gold(fname):
    fin = open (fname, 'r', encoding='utf-8')
    gold = {}
    for line in fin:
        fr, en = line.split("\t")
        fr_int = int(fr.replace('fr-',''))
        en_int = int(en.replace('en-',''))
    
        gold[str(fr_int) + "\t" + str(en_int)] = 0 # making french english pairs
    return (gold)

NameError: name 'load_gold' is not defined

In [103]:
### this is need only when predictions are in full matrix form
predicted_pairs = pred_pairs(top_idx)
print (len(predicted_pairs))

In [19]:
import cc_utils
import importlib
importlib.reload(cc_utils)
from cc_utils import *

In [5]:
gold = load_gold("/Users/hsajjad/Work/work/comparable-corpus/fr-en.training.gold")

In [6]:
pred_f = "/Users/hsajjad/Work/work/comparable-corpus/predictions-1best"
pred = load_prediction(pred_f)

In [10]:
# calculate precision, recall and fmeasure
score = calculateScore(pred, gold)
print (score)

p = 0.023474109330057304, r = 0.7023992956196347, f = 0.04542995444191344


In [317]:
# to calculate score of nbest list. This is to look at what is the best recall in our nbest list

In [20]:
pred_nbest = nbest_oracle("/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates")

dict

In [21]:
nbest_score = calculateOracleScore(pred_nbest, gold, 10)
print (nbest_score)

p = 0.027854815098170477, r = 0.8334800792427911, f = 0.053908029612756266


In [22]:
len(pred_nbest)

2718740

In [177]:
#### load MT file ## French to English
def load_mt(fname, mt_pairs, mt_idx):
    fin = open (fname, 'r', encoding='utf-8')
    count = 1
    for line in fin:
        fr, en = line.split("\t")
        mt_pairs[fr] = en
        mt_idx[count] = fr
        count += 1
    return

In [235]:
### loading MT output

mt_pairs = {}
mt_idx = {}
train_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/train-fr-en-mt-out"
load_mt(train_mt, mt_pairs, mt_idx)
len(mt_pairs)

270730

In [275]:
import pyter
def compareNbest_with_mt_ter(fr, en, mt_pairs):
    mt_en = mt_pairs[fr].split() #get corresponding translation
    en_wrd = en.split()
    return pyter.ter(en_wrd, mt_en)

In [ ]:
import pyter
def compareNbest_with_mt(fr, en, mt_pairs):
    mt_en = mt_pairs[fr] #get corresponding translation
    return nltk.translate.bleu_score.sentence_bleu([mt_en], en)

In [178]:
#### take nbest file and add an extra column with bleu score

def nbest_bleu_save(in_file, out_file, mt_pairs):
    
    nbest_out = open (out_file, 'w', encoding='utf-8')
    nbest = open (in_file, 'r', encoding='utf-8')

    curr_bleu = 0
    for line in nbest:
        line = line.rstrip()
        fr_id, fr, en_id, en = line.split("\t")

        curr_bleu = compareNbest_with_mt(fr, en, mt_pairs)

        nbest_out.write (line + "\t" + str(curr_bleu) + "\n")
    nbest_out.close()
    

In [276]:
#### take nbest file and add an extra column with bleu score

def nbest_ter_save(in_file, out_file, mt_pairs):
    
    nbest_out = open (out_file, 'w', encoding='utf-8')
    nbest = open (in_file, 'r', encoding='utf-8')

    curr_ter = 0
    for line in nbest:
        line = line.rstrip()
        fr_id, fr, en_id, en = line.split("\t")

        curr_ter = compareNbest_with_mt_ter(fr, en, mt_pairs)

        nbest_out.write (line + "\t" + str(curr_ter) + "\n")
    nbest_out.close()

In [277]:
# load MT output for train
train_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/train-fr-en-mt-out"
train_mt_pairs = {}
train_mt_idx = {}
load_mt(train_mt, train_mt_pairs, train_mt_idx)

### save nbest file with bleu scores on every line
train_nbest = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates"
train_nbest_ter = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-ter"
nbest_ter_save(train_nbest, train_nbest_ter, train_mt_pairs)



KeyboardInterrupt: 

In [179]:
train_nbest_ter = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-ter"
dic = getBestBleu(train_nbest_ter)

275683


/Users/hsajjad/anaconda/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/Users/hsajjad/anaconda/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/Users/hsajjad/anaconda/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [284]:
### load nbest file with bleu and filter out pairs
import sys

def getBestBleu(file_train_nbest_bleu):
    
    nbest_bleu = open (file_train_nbest_bleu, 'r', encoding='utf-8')

    best_bleu = -1
    best_str = ""
    dic = {}

    for line in nbest_bleu:
        line = line.rstrip()
        fr_id, fr, en_id, en, bleu = line.split("\t")

        if fr_id not in dic:
            dic[fr_id] = line
            best_bleu = bleu
            best_str = line
        elif bleu > best_bleu:
            best_bleu = bleu
            best_str = line
            dic[fr_id] = best_str
    return dic


In [285]:
### get best bleu dictionary and threshold on bleu

def filter_on_bleu(dic, threshold):
    final_list = {}
    eval_list = {}
    for key,value in dic.items():
        fr_id, fr, en_id, en, bleu = value.split("\t")
        if float(bleu) > threshold:
            final_list[key] = value
            fr_id = str(int(fr_id.replace('fr-','')))
            en_id = str(int(en_id.replace('en-','')))
            eval_list[fr_id + "\t" + en_id] = float(bleu)
    return eval_list

In [150]:
# load MT output for train
train_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/train-fr-en-mt-out"
train_mt_pairs = {}
train_mt_idx = {}
load_mt(train_mt, train_mt_pairs, train_mt_idx)

### save nbest file with bleu scores on every line
train_nbest = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates"
train_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-bleu"
nbest_bleu_save(train_nbest, train_nbest_bleu, train_mt_pairs)

train_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-bleu"
dic = getBestBleu(train_nbest_bleu)

In [320]:
threshold = 0
train_final = filter_on_bleu(dic, threshold)
calculateScore(train_final, gold)

7477 264396 1609
0.027501811507578907 0.8229143737618314 0.053224847753586824


In [236]:
# load MT output for test
test_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/test-fr-en-mt-out"
test_mt_pairs = {}
test_mt_idx = {}
load_mt(test_mt, test_mt_pairs, test_mt_idx)
print (len(test_mt_pairs))


275683


In [ ]:
## save nbest file with bleu scores on every line
test_nbest = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-testset"
test_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-testset-bleu"
nbest_bleu_save(test_nbest, test_nbest_bleu, test_mt_pairs)

In [225]:
test_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-testset-bleu"
test_pred = getBestBleu(test_nbest_bleu)
len(test_pred)

276833

In [259]:
threshold = 0.52
test_final = filter_on_bleu(test_pred, threshold)

In [260]:
def bucc_format(st, tag):
    st_rest = 9 - len(st)
    for i in range(st_rest):
        st = '0' + st
    st = tag + st
    return st
        
def output_format(pred_dic):
    format_dic = {}
    for key,value in pred_dic.items():
        fr, en = key.split("\t")
        fr = bucc_format(fr, "fr-")
        en = bucc_format(en, "en-")
        format_dic[fr + "\t" + en] = 0
    return format_dic

In [261]:
formatted_out = output_format(test_final)
test_output = open ("/Users/hsajjad/Work/work/comparable-corpus/submission/test-embedding-mt-0.52", 'w', encoding='utf-8')
for key,value in formatted_out.items():
    test_output.write(key)
    test_output.write("\n")
test_output.close()


In [224]:
calculateScore(eval_list, gold)

6511 1728 2575
0.7902658089573977 0.7165969623596742 0.7516305916305915


In [217]:
len(eval_list)

8239

In [ ]:
import nltk

In [89]:
ref = open ("/Users/hsajjad/Desktop/ref.ar", 'r', encoding='utf-8')
hypothesis = []
for line in ref:
    line = line.strip('\n')
    hypothesis.append(line.split(' '))

In [90]:
hyp = open("/Users/hsajjad/Desktop/output-ar", 'r', encoding='utf-8')
reference = []
for line in hyp:
    line = line.strip('\n')
    reference.append(line.split(' '))

In [91]:
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
cc = SmoothingFunction()
for i in range(len(hypothesis)):
    print(nltk.translate.bleu_score.sentence_bleu([reference[i]], hypothesis[i]))

NameError: name 'nltk' is not defined

In [75]:
from nltk.tokenize import sent_tokenize

In [76]:
par = "This is great. I am working."

In [78]:
sent_tokenize_list = sent_tokenize(par)

In [88]:
for i in range(len(sent_tokenize_list)):
    sent = sent_tokenize_list[i]
    words = print (sent.split())

['This', 'is', 'great.']
['I', 'am', 'working.']


In [209]:
import pyter
import importlib
importlib.reload(pyter)

<module 'pyter' from '/Users/hsajjad/anaconda/lib/python3.5/site-packages/pyter/__init__.py'>

In [274]:
hyp="this is going great".split()
ref="this is awesome what do you thing about it. we will enjoy this work".split()
pyter.ter(hyp, ref)

0.8571428571428571

In [281]:
#### QE task
qeFile = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-qe-predictions-train.sentids+qescores-filtered"
qeFile = "/Users/hsajjad/Work/work/comparable-corpus/eval-qe-bleu"
qeFile = "/Users/hsajjad/Work/work/comparable-corpus/eval-qe-0.5"

qpred ={}

qf = open(qeFile, 'r', encoding="utf-8")
for line in qf:
    line = line.rstrip()
    fr_id, en_id = line.split("\t")
    fr_id = str(int(fr_id.replace('fr-','')))
    en_id = str(int(en_id.replace('en-','')))
    qpred[fr_id + "\t" + en_id] = 0

In [282]:
len(qpred)

8739

In [283]:
calculateScore(qpred, gold)

5894 2845 3192
0.6744478773315025 0.6486902927580893 0.6613183730715287


In [ ]:
formatted_out = output_format(test_final)
test_output = open ("/Users/hsajjad/Work/work/comparable-corpus/submission/test-embedding-qe", 'w', encoding='utf-8')
for key,value in formatted_out.items():
    test_output.write(key)
    test_output.write("\n")
test_output.close()
